# Pos Padding _CBOW

In [ ]:
!git clone https://github.com/ntienhuy/MultiChannel

fatal: destination path 'MultiChannel' already exists and is not an empty directory.


In [ ]:
import numpy as np
import re
def load_data(folder_id):
    train_data = []
    train_labels = []    
    test_data = []
    test_labels = []
    for label in range(3):
        
        # read train data
        path = 'MultiChannel/data/VS/Data_Token/Fold_{}/train_nhan_{}.txt'.format(str(folder_id),str(label))
        f = open(path, "r",encoding='utf-8-sig')
        regex = re.compile('[-:()\",.!?=`~/+&$%^*#]')
        text = []
        for aline in f.readlines():
            aline = aline.lower()
            aline = regex.sub('', aline).strip()
            aline = re.sub(' +',' ',aline)
            text.append(aline)
        train_labels += [label]*len(text)
        train_data +=text
        print('label: {}, len: {}'.format(label, len(text)))

        # read test data
        path = 'MultiChannel/data/VS/Data_Token/Fold_{}/test_nhan_{}.txt'.format(str(folder_id),str(label))
        f = open(path, "r",encoding='utf-8-sig')
        regex = re.compile('[-:()\",.!?=`~/+&$%^*#]')
        text = []
        for aline in f.readlines():
            aline = aline.lower()
            aline = regex.sub('', aline).strip()
            aline = re.sub(' +',' ',aline)
            text.append(aline)
        test_labels += [label]*len(text)
        test_data += text

    return train_data, train_labels, test_data, test_labels

In [ ]:
folder_id = 1
train_data, train_labels, test_data, test_labels = load_data(folder_id=folder_id)

label: 0, len: 9461


label: 1, len: 7315


label: 2, len: 9303
{0, 1, 2}


In [ ]:
!pip install pytorch-nlp
from torchnlp.encoders.text import StaticTokenizerEncoder,pad_tensor, stack_and_pad_tensors

encoder = StaticTokenizerEncoder(train_data, tokenize=lambda s: s.split())

encoded_train_data = [encoder.encode(aline) for aline in train_data]
encoded_train_data = [pad_tensor(x, length=512) for x in encoded_train_data]
encoded_train_data = stack_and_pad_tensors(encoded_train_data)

encoded_test_data = [encoder.encode(aline) for aline in test_data]
encoded_test_data = [pad_tensor(x, length=512) for x in encoded_test_data]
encoded_test_data = stack_and_pad_tensors(encoded_test_data)

In [ ]:
!pip install  fasttext
import fasttext

In [ ]:
vocab_file = 'vocab_folder_{}.txt'.format(folder_id)
with open(vocab_file,"w") as f:
    for sentence in train_data:
        l = sentence.split()
        vocab+=l
        if len(l)>0:
            f.write(sentence+'\n')


In [ ]:
embedding_dim = 128

cbow_model = fasttext.train_unsupervised(input = vocab_file, dim = embedding_dim, model = "cbow")


In [ ]:
weights_matrix = np.zeros((encoder.vocab_size, embedding_dim))
word_ids = {}
idx = 0

for word in encoder.vocab:
    weights_matrix[idx] = cbow_model.get_word_vector(word)
    idx +=1
print(weights_matrix.shape)

def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.from_pretrained(torch.FloatTensor(weights_matrix))
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

(13204, 128)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
import time
torch.manual_seed(1)

In [ ]:
train_labels = torch.Tensor(train_labels)
dataset = torch.utils.data.TensorDataset(encoded_train_data[0], train_labels)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                           pin_memory =True, shuffle=True)

In [ ]:
test_labels = torch.Tensor(test_labels)
dataset = torch.utils.data.TensorDataset(encoded_test_data[0], test_labels)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                           pin_memory =True, shuffle=True)

In [ ]:
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)
class DevicedataLoader():
    def __init__(self,dl,device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)
print(device)
train_loader = DevicedataLoader(train_loader,device)
test_loader = DevicedataLoader(test_loader,device)

In [ ]:
class MyLSTM(nn.Module):
    def __init__(self, num_classes,  hidden_size, num_layers,weights_matrix):
        super(MyLSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        # self.input_size = input_size
        self.hidden_size = hidden_size
        # self.seq_length = seq_length
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix,True)

        # self.embedding = nn.Embedding( encoder.vocab_size, embedding_dim)

        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size,
                            num_layers = num_layers, bidirectional = False, batch_first = True)
        self.fc_1 = nn.Linear(self.hidden_size, 64)
        self.fc_2 = nn.Linear(64,num_classes)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self,x):
        # print(x)
        # x = x.transpose(0,1)
        len = torch.sum(x != 0, axis=1) -1
        # print(len)
        x = self.embedding(x)
        # print(x)
        # x, _ = self.lstm(x)
        
        x, (h_n, c_n)  = self.lstm(x)
        # x = h_n[-1]
        x = x[torch.arange(x.shape[0]),len,:]
        # x = x[:,-1,:]
        # x = x[-1]
        x = self.relu(self.fc_1(x))
        x = self.fc_2(x)
        # print(x)
        return x

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, test_loader):
    # model.load_state_dict(torch.load('folder_1_best.pth'))
    highest_acc = 0.75
    for epoch in range(1,n_epochs+1):
        model.train()
        loss_train = 0.0
        for vectors, labels in train_loader:
            vectors, labels = vectors.type(torch.LongTensor), labels.type(torch.LongTensor)
            output = model(vectors.to(device))
            optimizer.zero_grad()
            
            loss = F.cross_entropy(output, labels.to(device))
            
            loss.backward()
            optimizer.step()
            loss_train +=loss.item()
        correct = 0
        # break
        model.eval()
        with torch.no_grad():
            for data in test_loader:

                vec, label = data
                vec, label = vec.type(torch.LongTensor), label.type(torch.LongTensor)
                output = model(vec.to(device))
                predicted = torch.argmax(output,1)
                c = (predicted == label.to(device))
                correct += c.sum()
        if epoch == 1 or epoch % 1 == 0:
            print('Epoch {}, Training loss {}, Val accuracy {}'.format(
                epoch,
                loss_train / len(train_loader),
                correct / len(test_labels)))
        if correct/len(test_labels) >= highest_acc:
            highest_acc =  correct/len(test_labels)
            file_name = "folder_{}_pospadding_cbow.pth".format(folder_id)
            torch.save(model.state_dict(), file_name)

In [ ]:
learning_rate = 0.001
input_size = embedding_dim
hidden_size = embedding_dim
num_layers = 5
num_classes = 3

In [ ]:
lstm_model = MyLSTM(num_classes,  hidden_size, num_layers, weights_matrix) #our lstm class 
lstm_model.to('cuda:0')
optimizer = optim.Adam(lstm_model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
training_loop(
    n_epochs = 50,
    optimizer = optimizer,
    model = lstm_model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    test_loader = test_loader
)

Epoch 1, Training loss 0.9355479627847672, Val accuracy 0.577646017074585


Epoch 2, Training loss 0.7601457413505105, Val accuracy 0.5895054936408997


Epoch 3, Training loss 0.6479029144872638, Val accuracy 0.6521593332290649


Epoch 4, Training loss 0.5420408661970321, Val accuracy 0.6405236124992371


Epoch 5, Training loss 0.43894272136921975, Val accuracy 0.6729693412780762


...

Epoch 12, Training loss 0.120025334932257, Val accuracy 0.6935555934906006


Epoch 13, Training loss 0.09664906490498278, Val accuracy 0.6966882944107056


Epoch 14, Training loss 0.09389084155840215, Val accuracy 0.7105616331100464


Epoch 15, Training loss 0.08841602266286774, Val accuracy 0.7140299677848816


Epoch 16, Training loss 0.08123846209377471, Val accuracy 0.7100022435188293


# pos-padding cbow
- folder1 acc: 0.7414
- folder2 acc: 0.7861
- folder3 acc: 0.8224
- folder4 acc: 0.8209
- folder5 acc: 0.8194

# pre-padding cbow
- folder1 acc: 0.7355
- folder2 acc: 0.7714
- folder3 acc: 0.8085
- folder4 acc: 0.7714
- folder5 acc: 0.7195

# folder 1:
- pre_padding, CBOW: 0.7355
- post_padding, CBOW: 0.7417
- pre-padding, BPEmbedding: 0.7417
- pos-padding, BPEmbedding: 0.75766
# folder 2:
- pre_padding, CBOW: 0.7714
- post_padding, CBOW: 0.7861
- pre-padding, BPEmbedding: 0.7659
- post_padding, BPEmbedding: 0.7941
